In [1]:
import numpy as np
import osmnx as ox
import networkx as nx

import seaborn as sns
import matplotlib.pyplot as plt

import random
import pickle
import pymde
from sklearn.manifold import MDS, Isomap, TSNE, LocallyLinearEmbedding, SpectralEmbedding

import osqp
import mlrfit as mf
import lrrouting as ldr
from scipy import sparse

import cvxpy as cp
import time 

/Users/parshakova.tanya/opt/anaconda3/envs/routing/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
np.random.seed(1001)
random.seed(1001)

#  Matrix definition

In [3]:
rank = 20

mtype = "small_world"
n = 1000

G = nx.connected_watts_strogatz_graph(n, k=10, p=0.1)
G.remove_edges_from(nx.selfloop_edges(G))

n = G.number_of_nodes()
print(f"{n=}, {G.number_of_edges()=}")

for u, v in G.edges():
    G[u][v]['weight'] = np.random.rand() * 10

Adj, Dist = ldr.nx_graph_to_matrices(G)
A = Dist

n=1000, G.number_of_edges()=5000
in  degrees: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 2, 7: 5, 8: 46, 9: 237, 10: 448, 11: 190, 12: 56, 13: 14, 14: 2}
out degrees: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 2, 7: 5, 8: 46, 9: 237, 10: 448, 11: 190, 12: 56, 13: 14, 14: 2}


In [4]:
assert nx.is_connected(G)
np.histogram(Dist.flatten(), bins=5, density=True)

(array([8.73485171e-03, 7.97163579e-02, 1.01414309e-01, 8.86130545e-03,
        9.98109705e-05]),
 array([ 0.        ,  5.02950725, 10.0590145 , 15.08852175, 20.118029  ,
        25.14753626]))

In [5]:
adjacency_list = ldr.adjacency_list(Adj)
sources, targets = ldr.st_pairs(n, Dist, 1020)
M = min(1000, sources.size)
sources = sources[:M]
targets = targets[:M]

In [6]:
PSD = False
w_min = A[A>0].min()
rt_max_iters = min(int(5*A.max()/w_min), (10**4) // 2)
symm = np.allclose(A, A.T)
print(f"{symm=}")
filename = "%s_r%d_%d"%(mtype, rank, n)

symm=True


In [7]:
np.histogram(Adj[Adj>0], bins=5, density=True)

(array([0.10261409, 0.09671328, 0.09551311, 0.10411429, 0.10111388]),
 array([8.50688893e-04, 2.00057610e+00, 4.00030151e+00, 6.00002692e+00,
        7.99975233e+00, 9.99947774e+00]))

In [8]:
fraction_of_nodes = 0.1
pi = np.random.permutation(n)[:int(n * fraction_of_nodes)]

In [9]:
info = {} 

# $\ell_2$-norm based embeddings

In [10]:
# check if the method is a block coordinate descent 
max_iter = 10

durations_l2 = {"gi":[], "bcd":[], "cycle":[]}
for ord in [2]:
    norm_ord = lambda a: ldr.norm_subgradient(a, ord=ord)
    X_emb = np.random.randn(n, rank)
    losses = [ldr.obj_distortion_function(Dist, X_emb, ord)]
    print(f"{ord=}, distortion={losses[-1]}")
    for t in range(max_iter):
        start_cycle_time = time.time()
        for i in pi:
            start = time.time()
            Dist_i = Dist[i]
            g_i = np.apply_along_axis(norm_ord, 1, np.repeat(X_emb[i:i+1], n, axis=0) - X_emb)
            durations_l2["gi"] += [time.time() - start]
            start = time.time()
            x_i, obj_i = ldr.convex_concave(Dist_i, i, X_emb, ord, solver=cp.CLARABEL, g_i=g_i)
            durations_l2["bcd"] += [time.time() - start]
            x_i2, obj_i2 = ldr.bcd_l2_single(n, rank, i, Dist_i, X_emb, g_i)
            assert np.allclose(obj_i, obj_i2), print(f"{obj_i=}, {obj_i2=}")

            losses += [ldr.obj_distortion_function(Dist, X_emb, ord)]
            X_emb[i] = x_i
        durations_l2["cycle"] += [time.time()-start_cycle_time]
        if t%10 == 0 or t == max_iter-1:
            print(f"{t=}, distortion={losses[-1]}, cycle_time={time.time()-start_cycle_time}")

    assert (np.diff(losses) < 1e-6).all(), print("it is not a descent method")
    assert X_emb.shape == (n, rank)
    linf_dar = ldr.construct_node_embedding_graph(X_emb, adjacency_list)
    info["nemb_l%s"%str(ord)] = {'ratios' : ldr.subopt_ratios(linf_dar, Dist, sources, targets)}

ord=2, distortion=52.20712737340071


/Users/parshakova.tanya/opt/anaconda3/envs/routing/lib/python3.9/site-packages/cvxpy/problems/problem.py:1403: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


t=0, distortion=47.00816872229839, cycle_time=33.4904510974884
t=9, distortion=45.35077310176677, cycle_time=34.31567311286926
median_stretch=2700.8%, mean_stretch=3808.1%
%[ratio<2] = 5.60%, %[ratio<1.2] = 3.10%, %[ratio=1.] = 1.80%


In [11]:
for k, val in durations_l2.items():
    print(f"{k}, {np.mean(np.array(val))}+/-{np.std(np.array(val))}")

gi, 0.005502525091171264+/-0.0008349427259771
bcd, 0.20742909693717956+/-0.013215156129425252
cycle, 34.55622508525848+/-0.8292628584784719


In [12]:
%timeit ldr.bcd_l2_single(n, rank, i, Dist_i, X_emb, g_i)

1.24 ms ± 69.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [13]:
%timeit ldr.convex_concave(Dist_i, i, X_emb, ord, solver=cp.CLARABEL, g_i=g_i)

205 ms ± 3.19 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# $\ell_\infty$-norm based embeddings

In [14]:
# compare CVXPY + Clarabel vs calling directly OSQP solver
max_iter = 2
durations_linf = {"gi":[], "osqp":[], "cvxpy":[], "cycle":[]}
for ord in [np.inf]:
    norm_ord = lambda a: ldr.norm_subgradient(a, ord=ord)
    X_emb = np.random.randn(n, rank)
    losses = [ldr.obj_distortion_function(Dist, X_emb, ord)]
    print(f"{ord=}, distortion={losses[-1]}")
    P, M0 = ldr.matrices_bcd_linf_single(n, rank)
    for t in range(max_iter):
        start_cycle_time = time.time()
        for i in pi:
            Dist_i = Dist[i]
            start = time.time()
            g_i = np.apply_along_axis(norm_ord, 1, np.repeat(X_emb[i:i+1], n, axis=0) - X_emb)
            durations_linf["gi"] += [time.time() - start]
            start = time.time()
            x_i, obj_i = ldr.convex_concave(Dist_i, i, X_emb, ord, solver=cp.CLARABEL, g_i=g_i)
            durations_linf["cvxpy"] += [time.time() - start]
            start = time.time()
            x_i2, obj_i2 = ldr.bcd_linf_single(n, rank, i, Dist_i, X_emb, g_i, M0, eps_abs=1e-3, eps_rel=1e-3, max_iter=4000)
            durations_linf["osqp"] += [time.time() - start]

            losses += [ldr.obj_distortion_function(Dist, X_emb, ord)]
            X_emb[i] = x_i
        durations_linf["cycle"] += [time.time()-start_cycle_time]
        if t%1 == 0 or t == max_iter-1:
            print(f"{t=}, distortion={losses[-1]}, cycle_time={time.time()-start_cycle_time}")

    assert (np.diff(losses) < 1e-6).all(), print("it is not a descent method")
    assert X_emb.shape == (n, rank)
    linf_dar = ldr.construct_node_embedding_graph(X_emb, adjacency_list)
    info["nemb_l%s"%str(ord)] = {'ratios' : ldr.subopt_ratios(linf_dar, Dist, sources, targets)}

ord=inf, distortion=124.28777305097695
t=0, distortion=104.98890516017494, cycle_time=156.69867491722107
t=1, distortion=104.63253218900468, cycle_time=147.84865379333496
median_stretch=2527.6%, mean_stretch=3880.4%
%[ratio<2] = 5.20%, %[ratio<1.2] = 2.70%, %[ratio=1.] = 1.60%


In [15]:
for k, val in durations_linf.items():
    print(f"{k}, {np.mean(np.array(val))}+/-{np.std(np.array(val))}")

gi, 0.003687340021133423+/-0.0007986835126807514
osqp, 0.8440155684947968+/-0.2879381610682616
cvxpy, 0.495924813747406+/-0.03595807217873438
cycle, 152.2736474275589+/-4.425012469291687


In [16]:
233*(0.0057397+0.73)/(0.0057397+1.34+0.73)

82.58614993970585

In [17]:
%timeit ldr.obj_distortion_function(Dist, X_emb, ord)

182 ms ± 15.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%timeit ldr.bcd_linf_single(n, rank, i, Dist_i, X_emb, g_i, M0)

845 ms ± 4.21 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
%timeit ldr.convex_concave(Dist_i, i, X_emb, ord=np.inf, solver=cp.CLARABEL, g_i=g_i)

490 ms ± 5.61 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
258/0.223

1156.9506726457398

In [21]:
M = []
for j in range(n):
    e_j = np.zeros(n)
    e_j[j] = 1
    ones_e_j = sparse.coo_array(np.tile(e_j, (rank, 1)))
    block = sparse.bmat([[-ones_e_j, sparse.eye(rank)],
                        [ones_e_j, sparse.eye(rank)]], format='csc')
    M += [block]
M1 = sparse.vstack(M, format='csc')


M = []
for j in range(n):
    ones_e_j = np.zeros((rank, n))
    ones_e_j[:, j] = 1
    block = np.block([[-ones_e_j, np.eye(rank)],
                        [ones_e_j, np.eye(rank)]],)
    M += [block]
M = np.concatenate(M, axis=0)
M2 = sparse.csc_matrix(M)
assert np.allclose(M1.toarray(), M2.toarray())
print("PASSED")

PASSED


In [22]:
for _ in range(10):
    X_emb = np.random.randn(n, rank)
    u, l = [], []
    for j in range(n):
        u += [X_emb[j], np.inf * np.ones(rank)]
        l += [-np.inf * np.ones(rank), X_emb[j]]
    u += [np.zeros(1)]
    l += [np.zeros(1)]
    u0 = np.concatenate(u, axis=0)
    l0 = np.concatenate(l, axis=0)

    u = np.concatenate([X_emb, np.full((n,rank ), np.inf)], axis=1).flatten(order='C')
    l = np.concatenate([np.full((n,rank ), -np.inf), X_emb], axis=1).flatten(order='C')
    u = np.concatenate([u, np.zeros(1)], axis=0)
    l = np.concatenate([l, np.zeros(1)], axis=0)
    assert np.allclose(u0, u) and np.allclose(l0, l)
print("PASSED")

PASSED


In [23]:
X_emb = np.random.randn(n, rank)
i = 20
norm_ord = lambda a: ldr.norm_subgradient(a, ord=2)
g_i = np.apply_along_axis(norm_ord, 1, np.repeat(X_emb[i:i+1], n, axis=0) - X_emb)
assert np.allclose(Dist[i][:, np.newaxis] * g_i, np.diag(Dist[i]) @ g_i)